## Reddit Text Classification

In [2]:
import pandas as pd

First we import the training data from the given .csv file.

In [8]:
# Import the dataset
df = pd.read_csv('train.csv', delimiter=',', header=None, encoding='latin-1')

# Remove the first row
df = df.iloc[1:]

# Rename the columns
df.columns = ['body', 'subreddit']

# Print the first 5 rows
df.head()

,body,subreddit
1,"From what I have seen in my neighbourhood, too...",Toronto
2,Copenhagen broke ground on their new subway li...,Toronto
3,I work in big 5 headquarters. Lmao good luck b...,Toronto
4,"If the person that did this is just a psycho, ...",Toronto
5,re: Police managing traffic - I regularly see ...,Toronto


Some preprocessing is helpful prior to tokenization. This includes expanding contracted words, removing stopwords (a, an the), and converting all text to lowercase.

In [9]:
import contractions

# Expand the contracted words
df['body'] = 

,body,subreddit
1,"[From, what, I, have, seen, in, my, neighbourh...",Toronto
2,"[Copenhagen, broke, ground, on, their, new, su...",Toronto
3,"[I, work, in, big, 5, headquarters., Lmao, goo...",Toronto
4,"[If, the, person, that, did, this, is, just, a...",Toronto
5,"[re:, Police, managing, traffic, -, I, regular...",Toronto
